In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
credit_data=pd.read_csv("../input/CC GENERAL.csv")
credit_data.shape
credit_data.isnull().sum()
credit_data.dtypes
credit_data.drop(['CUST_ID'], axis=1, inplace=True)
from sklearn.preprocessing import Imputer
mean_imputer=Imputer(strategy='mean')
imputed_credit_data=pd.DataFrame(mean_imputer.fit_transform(credit_data),columns=credit_data.columns)

# Any results you write to the current directory are saved as output.

In [ ]:
imputed_credit_data.isnull().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler
standardizer=StandardScaler()
std_x=standardizer.fit_transform(imputed_credit_data)
std_credit_data=pd.DataFrame(std_x,columns=imputed_credit_data.columns)


In [ ]:
std_credit_data.describe

from sklearn.cluster import KMeans
kmeans_object=KMeans(n_clusters=5,random_state=1234)
kmeans_cluster=kmeans_object.fit_predict(std_credit_data)
kmeans_object.labels_
cluster_centroids = pd.DataFrame(standardizer.inverse_transform(kmeans_object.cluster_centers_),columns=credit_data.columns)
cluster_centroids

In [ ]:
wss={}
for k in range(1,20):
    kmeans_loop=KMeans(n_clusters=k,n_init=30,n_jobs=2,random_state=1000,verbose=0).fit(std_credit_data)
    clusters = kmeans_loop.labels_
    wss[k] = kmeans_loop.inertia_ # Inertia: Sum of squared distances of samples to their closest cluster center
wss

In [ ]:
wss.keys()

In [ ]:
wss.values()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
# %matplotlib notebook

plt.plot(wss.keys(),wss.values(),marker='o')

# plt.plot(list(wss.keys()),list(wss.values()))
plt.grid()
plt.xlabel('Number of clusters')
plt.ylabel('Total within sum of squares')
plt.show()


In [ ]:
from __future__ import print_function

# from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# plt.figure()

print(__doc__)

### Just change this to your dataframe
X_matrix = std_credit_data.as_matrix()

range_n_clusters = [3, 4, 5, 6, 7, 8, 9, 10, 11]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 1 column
    fig, (ax1) = plt.subplots(1, 1)
    fig.set_size_inches(9, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, +1 
    
    ax1.set_xlim([-0.25, .5])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    
    ax1.set_ylim([0, len(X_matrix) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X_matrix)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X_matrix, cluster_labels)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X_matrix, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    plt.suptitle(("For %d clusters, silhouette avg coeff = %f " % (n_clusters,silhouette_avg)),
                 fontsize=14, fontweight='bold')
    plt.show()

In [ ]:
best_kmeans = KMeans(n_clusters=3, random_state=1240)
best_kmeans.fit(std_credit_data)
best_kmeans_labels = best_kmeans.predict(std_credit_data)
kmeans_results=pd.DataFrame(best_kmeans_labels)
kmeans_results.head()